## In this Python Notebook, we will be:
- ✅ Load the dataset
- ✅ Handle missing values
- ✅ Convert data types
- ✅ Apply rule-based fraud risk scoring
- ✅ Extract new features (transaction frequency per user)
- ✅ Save the cleaned dataset for Power BI

In [2]:
import pandas as pd

transactions = pd.read_csv("../data/raw/transactions.csv")
transactions.head(5)

,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate
0,TX000001,AC00128,14.09,2023-04-11 16:29:14,Debit,San Diego,D000380,162.198.218.92,M015,ATM,70,Doctor,81,1,5112.21,2024-11-04 08:08:08
1,TX000002,AC00455,376.24,2023-06-27 16:44:19,Debit,Houston,D000051,13.149.61.4,M052,ATM,68,Doctor,141,1,13758.91,2024-11-04 08:09:35
2,TX000003,AC00019,126.29,2023-07-10 18:16:08,Debit,Mesa,D000235,215.97.143.157,M009,Online,19,Student,56,1,1122.35,2024-11-04 08:07:04
3,TX000004,AC00070,184.50,2023-05-05 16:32:11,Debit,Raleigh,D000187,200.13.225.150,M002,Online,26,Student,25,1,8569.06,2024-11-04 08:09:06
4,TX000005,AC00411,13.45,2023-10-16 17:51:24,Credit,Atlanta,D000308,65.164.3.100,M091,Online,26,Student,198,1,7429.40,2024-11-04 08:06:39


In [3]:
# Since in this dataset, the date comes in timestamp format, we're converting the timestamp column to datetime

if "TransactionDate" in transactions.columns:
    transactions["TransactionDate"] = pd.to_datetime(transactions["TransactionDate"])

transactions['TransactionDate'].head()

0   2023-04-11 16:29:14
1   2023-06-27 16:44:19
2   2023-07-10 18:16:08
3   2023-05-05 16:32:11
4   2023-10-16 17:51:24
Name: TransactionDate, dtype: datetime64[ns]

In [4]:
# Handling missing values
transactions.fillna(transactions.median(numeric_only=True), inplace=True)  # Fill numeric NaNs with median
transactions.fillna("Unknown", inplace=True)  # Fill categorical NaNs with 'Unknown'

### Initializing Rule-Based Transaction Scoring

1️⃣ Define Risk Factors: adding points based on conditions like:
- **Transaction Amount**: Large amounts are riskier.
- **Transaction Time**: Late-night transactions are suspicious.
- **Transaction Frequency**: Users making multiple quick transactions might be fraudsters.
- **Merchant Category**: Some categories (e.g., luxury, digital goods) might be riskier.

2️⃣ Compute a Risk Score
We'll assign weights to these factors and normalize the score between 0 (low risk) and 1 (high risk).

3️⃣ Save the Updated Dataset
The new column will be saved as `fraud_risk_score`.

In [7]:
transactions["FraudRiskScore"] = 0  # Initialize risk score
transactions["FraudRiskScore"].head()

0    0
1    0
2    0
3    0
4    0
Name: FraudRiskScore, dtype: int64